In [56]:
import os

from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from tqdm import tqdm

html_directory = r"./DATA/财务报表HTML/"
html_files = [file for file in os.listdir(html_directory) if file.endswith(".html")]
for html_file in tqdm(html_files):
    with open(html_directory + html_file, 'r', encoding='gbk') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'lxml')  # Parse the HTML using BeautifulSoup
    tables_with_class = soup.find_all('table',
                                      class_='ntable ntable-table-fixed')
    # 第一个表的列索引是年份，表中内容为空，注意第一列无意义
    try:
        table_head = tables_with_class[0]
    except IndexError:
        print(f'{html_file} 表头年份为空')
        continue
    df_head = pd.read_html(StringIO(str(table_head)))[0].iloc[:, 1:]
    years = [year.split("年")[0] for year in df_head.columns]
    # 第二个表是具体的数据
    try:
        table = tables_with_class[1]
    except IndexError:
        print(f'{html_file} 数据内容为空')
        continue
    df = pd.read_html(StringIO(str(table)))[0]
    df = df.dropna(subset=df.columns[1:], how='all')  # 去除全是NULL的行
    # 行索引为’同比%‘时，修改为具体指标+同比
    # for i in range(df.shape[1]):
    #     if "同比" in df.iloc[i, 0]:
    #         df.iloc[i, 0] = df.iloc[i - 1, 0] + df.iloc[i, 0]
    for i in range(1, df.shape[0]):
        if "同比" in str(df.index[i]):
            df.index[i] = df.index[i - 1] + df.index[i]

    # 第一列为行索引
    df.set_index(df.columns[0], inplace=True)
    df.columns = years
    csv_file = r"./DATA/财务报表CSV/" + html_file.split(".html")[0] + ".csv"
    df.to_csv(csv_file, index=True)

  4%|▎         | 6/167 [00:00<00:05, 28.15it/s]

上海仁杰河滨园房地产有限公司.html 表头年份为空


 19%|█▊        | 31/167 [00:00<00:02, 49.75it/s]

中山市长健医用包装材料有限公司.html 表头年份为空


 66%|██████▋   | 111/167 [00:02<00:01, 41.71it/s]

济南四建（集团）有限责任公司.html 表头年份为空


100%|██████████| 167/167 [00:04<00:00, 37.71it/s]
